# FO linear CV model KF
## Model
### States
$$
\begin{align*}
\bold{r}_{B/N}^n = \begin{bmatrix} N \\ E \end{bmatrix} \\
\bold{\Theta} = \begin{bmatrix} \psi \end{bmatrix} \\
\bold{v}_{\mathcal{B}/\mathcal{N}}^b = \begin{bmatrix} u \\ v \end{bmatrix} \\
\bold{\omega}^b_{\mathcal{B}/\mathcal{N}} = \begin{bmatrix} r \end{bmatrix} \\
\bold{\eta} = \begin{bmatrix} \bold{r}_{B/N}^n \\ \bold{\Theta} \\ \end{bmatrix}\\
\bold{\nu} = \begin{bmatrix} \bold{v}_{\mathcal{B}\mathcal{N}}^b \\ \bold{\omega}^b_{\mathcal{B}/\mathcal{N}} \\ \end{bmatrix}\\
\bold{x} := \begin{bmatrix} \bold{\eta} \\ \bold{\nu} \end{bmatrix} \\
\implies \bold{x} = \begin{bmatrix} N \\ E \\ \psi \\ u \\ v \\ r \end{bmatrix} \\
\end{align*}
$$
Where
- $N$ is translational displacement in the North direction in local NED frame.
- $E$ is translational displacement in the East direction in local NED frame.
- $\psi$ is angular displacement about the Down direction in local NED frame.
- $u$ is translational speed in the X direction in the body fixed frame.
- $v$ is translational speed in the Y direction in the body fixed frame.
- $r$ is angular rate about the Z direction in the body fixed frame.

### Inputs
$$
\begin{align*}
\bold{u} = \begin{bmatrix} X \\ Y \\ N\end{bmatrix} \\
\end{align*}
$$
Where
- $X$ is an **unknown** translational acceleration in the surge direction in the body frame.
- $Y$ is an **unknown** translational acceleration in the sway direction in the body frame.
- $N$ is an **unknown** rotational acceleration about the z axis in the body frame.

### Measurements
$$
\begin{align*}
\bold{z} = \begin{bmatrix} \text{GPS}_N \\ \text{GPS}_E \\ \text{GPS}_\psi\end{bmatrix} \\
\end{align*}
$$
Where
- $\text{GPS}_N$ is the North component of a GPS measurement represented in local NED frame.
- $\text{GPS}_E$ is the East component of a GPS measurement represented in local NED frame.
- $\text{GPS}_\psi$ is the rotation about the $\vec{e}_3$ direction measured by the GPS represented in local NED frame.

### Modeling
#### State transition
For $\eta$
$$
\begin{align*}
\dot{\eta} = J_k(\eta) \nu \\
J_k(\eta) = \begin{bmatrix}R^n_n(\Theta) & \vec{0}\\\vec{0} & T_k(\Theta)\end{bmatrix} \\
R^n_b(\Theta) = \begin{bmatrix} cos(psi) & -sin(psi) & 0 \\ sin(psi)& cos(psi)& 0 \\ 0&0&1\end{bmatrix} \\
T_k(\Theta) = \begin{bmatrix}1 & sin(\phi)tan(\theta) & cos(\phi)tan(\theta)\\ 0 & cos(\phi) & -sin(\phi) \\ 0 & \frac{sin(\psi)}{cos(\theta)} & \frac{cos(\phi)}{cos(\theta)}\end{bmatrix} \\
\theta = \phi = 0 \\

\implies \eta_{k+1} = \eta + T (J_k(\eta) \nu)\\
\end{align*}
$$

For $\nu$
$$
\begin{align*}
\dot{\nu} = f(\bold{u}) \\
\dot{\nu} = T\bold{u}\\
\implies \nu_{k+1} = \nu + T\bold{u}\\

\end{align*}
$$

#### Measurement equation
$$
\begin{align*}
z_k = H \bold{x}_k\\
\implies \begin{bmatrix} \text{GPS}_N \\ \text{GPS}_E \\ \text{GPS}_\psi\end{bmatrix} = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0\\ 0 & 1 & 0 & 0 & 0 & 0 \\0 & 0 & 1 & 0 & 0 & 0 \end{bmatrix} \begin{bmatrix} N \\ E \\ \psi \\ u \\ v \\ r \end{bmatrix}\\
\end{align*}
$$


In [39]:
import sympy as sp

# Define a symbol for time
t = sp.symbols('t')

# Define state variables as functions of time
N = sp.Function('N')(t)
E = sp.Function('E')(t)
D = sp.Function('D')(t)
phi = sp.Function('phi')(t)
theta = sp.Function('theta')(t)
psi = sp.Function('psi')(t)
u = sp.Function('u')(t)
v = sp.Function('v')(t)
w = sp.Function('w')(t)
p = sp.Function('p')(t)
q = sp.Function('q')(t)
r = sp.Function('r')(t)

# Define symbol for time step
T = sp.symbols('T')

rBNn = sp.Matrix([[N], [E], [D]])
Theta = sp.Matrix([[phi], [theta], [psi]])
eta  = sp.Matrix([[rBNn], [Theta]])

omegaBNb = sp.Matrix([[p], [q],[r]])
vBNb = sp.Matrix([[u], [v], [w]])
nu  = sp.Matrix([[vBNb], [omegaBNb]])

x = sp.Matrix([[eta], [nu]])
sp.pprint(x, use_unicode=True)

⎡N(t)⎤
⎢    ⎥
⎢E(t)⎥
⎢    ⎥
⎢D(t)⎥
⎢    ⎥
⎢φ(t)⎥
⎢    ⎥
⎢θ(t)⎥
⎢    ⎥
⎢ψ(t)⎥
⎢    ⎥
⎢u(t)⎥
⎢    ⎥
⎢v(t)⎥
⎢    ⎥
⎢w(t)⎥
⎢    ⎥
⎢p(t)⎥
⎢    ⎥
⎢q(t)⎥
⎢    ⎥
⎣r(t)⎦


In [40]:
# Rotation about the X-axis (Roll)
def Rx_sp(phi):
    return sp.Matrix([
        [1, 0, 0],
        [0, sp.cos(phi), -sp.sin(phi)],
        [0, sp.sin(phi),  sp.cos(phi)]
    ])


# Rotation about the Y-axis (Pitch)
def Ry_sp(theta):
    return sp.Matrix([
        [ sp.cos(theta), 0, sp.sin(theta)],
        [0, 1, 0],
        [-sp.sin(theta), 0, sp.cos(theta)]
    ])


# Rotation about the Z-axis (Yaw)
def Rz_sp(psi):
    return sp.Matrix([
        [sp.cos(psi), -sp.sin(psi), 0],
        [sp.sin(psi),  sp.cos(psi), 0],
        [0, 0, 1]
    ])


def R_zyx_sym(phi, theta, psi):
    return Rz_sp(psi) * Ry_sp(theta) * Rx_sp(phi)


In [41]:
# --- Rigid body rotation from body to inertial frame ---
Rnb = Rz_sp(psi) * Ry_sp(theta) * Rx_sp(phi)
Rnb

Matrix([
[cos(psi(t))*cos(theta(t)), sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t))],
[sin(psi(t))*cos(theta(t)), sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t))],
[           -sin(theta(t)),                                       sin(phi(t))*cos(theta(t)),                                        cos(phi(t))*cos(theta(t))]])

In [42]:
# --- Rigid body rotation from body to inertial frame ---
Tk = sp.Matrix([[1, sp.sin(phi) * sp.tan(theta), sp.cos(phi) * sp.tan(theta)],
                       [0, sp.cos(phi), -sp.sin(phi)],
                       [0, sp.sin(psi) / sp.cos(theta), sp.cos(phi) / sp.cos(theta)]])

Tk

Matrix([
[1, sin(phi(t))*tan(theta(t)), cos(phi(t))*tan(theta(t))],
[0,               cos(phi(t)),              -sin(phi(t))],
[0, sin(psi(t))/cos(theta(t)), cos(phi(t))/cos(theta(t))]])

In [43]:
# --- Velocity transformation matrix ---
Jk = sp.Matrix([[Rnb, sp.zeros(3,3)], [sp.zeros(3,3), Tk]])
Jk

Matrix([
[cos(psi(t))*cos(theta(t)), sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)),  sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)), 0,                         0,                         0],
[sin(psi(t))*cos(theta(t)), sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)), -sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)), 0,                         0,                         0],
[           -sin(theta(t)),                                       sin(phi(t))*cos(theta(t)),                                        cos(phi(t))*cos(theta(t)), 0,                         0,                         0],
[                        0,                                                               0,                                                                0, 1, sin(phi(t))*tan(theta(t)), cos(phi(t))*tan(theta(t))],
[                        0,                                                               0,                               

In [44]:
d_eta = Jk * nu
d_eta

Matrix([
[ (sin(phi(t))*sin(psi(t)) + sin(theta(t))*cos(phi(t))*cos(psi(t)))*w(t) + (sin(phi(t))*sin(theta(t))*cos(psi(t)) - sin(psi(t))*cos(phi(t)))*v(t) + u(t)*cos(psi(t))*cos(theta(t))],
[(-sin(phi(t))*cos(psi(t)) + sin(psi(t))*sin(theta(t))*cos(phi(t)))*w(t) + (sin(phi(t))*sin(psi(t))*sin(theta(t)) + cos(phi(t))*cos(psi(t)))*v(t) + u(t)*sin(psi(t))*cos(theta(t))],
[                                                                                            -u(t)*sin(theta(t)) + v(t)*sin(phi(t))*cos(theta(t)) + w(t)*cos(phi(t))*cos(theta(t))],
[                                                                                                           p(t) + q(t)*sin(phi(t))*tan(theta(t)) + r(t)*cos(phi(t))*tan(theta(t))],
[                                                                                                                                              q(t)*cos(phi(t)) - r(t)*sin(phi(t))],
[                                                                                     

In [45]:
# --- Skew symmetric matrix ---
def skew(vec):
    return sp.Matrix([
        [0, -vec[2], vec[1]],
        [vec[2], 0, -vec[0]],
        [-vec[1], vec[0], 0]
    ])

In [46]:
# General center of gravity vector from body origin to CG
m, xg, yg, zg = sp.symbols('m x_g y_g z_g')
r_g = sp.Matrix([xg, yg, zg])

# Skew-symmetric matrix of CG
S_rg = skew(r_g)

# Full symbolic 3x3 inertia matrix
Ixx, Iyy, Izz, Ixy, Ixz, Iyz = sp.symbols('I_xx I_yy I_zz I_xy I_xz I_yz')
I_body = sp.Matrix([
    [Ixx, -Ixy, -Ixz],
    [-Ixy, Iyy, -Iyz],
    [-Ixz, -Iyz, Izz]
])

# Full rigid-body mass matrix with coupling terms
MRB = sp.BlockMatrix([
    [m * sp.eye(3), -m*S_rg],
    [m*S_rg,      I_body]
]).as_explicit()
MRB

Matrix([
[     m,      0,      0,      0,  m*z_g, -m*y_g],
[     0,      m,      0, -m*z_g,      0,  m*x_g],
[     0,      0,      m,  m*y_g, -m*x_g,      0],
[     0, -m*z_g,  m*y_g,   I_xx,  -I_xy,  -I_xz],
[ m*z_g,      0, -m*x_g,  -I_xy,   I_yy,  -I_yz],
[-m*y_g,  m*x_g,      0,  -I_xz,  -I_yz,   I_zz]])

In [47]:
# --- Coriolis matrix ---
S_v = skew(vBNb)
S_omega = skew(omegaBNb)

# CRB = sp.BlockMatrix([
#     [sp.zeros(3, 3), -m * S_v],
#     [-m * S_v, -skew(I_body * omegaBNb)]
# ]).as_explicit()

CRB = sp.BlockMatrix([
    [m*S_omega, -m*S_v*S_rg],
    [m*S_rg*S_v, -skew(I_body * omegaBNb)]
]).as_explicit()


CRB

Matrix([
[                       0,                  -m*r(t),                   m*q(t),            m*y_g*v(t) + m*z_g*w(t),                        -m*x_g*v(t),                       -m*x_g*w(t)],
[                  m*r(t),                        0,                  -m*p(t),                        -m*y_g*u(t),            m*x_g*u(t) + m*z_g*w(t),                       -m*y_g*w(t)],
[                 -m*q(t),                   m*p(t),                        0,                        -m*z_g*u(t),                        -m*z_g*v(t),           m*x_g*u(t) + m*y_g*v(t)],
[-m*y_g*v(t) - m*z_g*w(t),               m*y_g*u(t),               m*z_g*u(t),                                  0, -I_xz*p(t) - I_yz*q(t) + I_zz*r(t), I_xy*p(t) - I_yy*q(t) + I_yz*r(t)],
[              m*x_g*v(t), -m*x_g*u(t) - m*z_g*w(t),               m*z_g*v(t),  I_xz*p(t) + I_yz*q(t) - I_zz*r(t),                                  0, I_xx*p(t) - I_xy*q(t) - I_xz*r(t)],
[              m*x_g*w(t),               m*y_g*w(t), -m*

In [48]:
# --- Gravity vector ---
g, z_g = sp.symbols('g z_g')
g_eta = sp.Matrix([
    [0],
    [0],
    [m * g],
    [m * g * z_g * sp.sin(theta)],
    [-m * g * z_g * sp.sin(phi) * sp.cos(theta)],
    [0]
])
g_eta

Matrix([
[                                 0],
[                                 0],
[                               g*m],
[             g*m*z_g*sin(theta(t))],
[-g*m*z_g*sin(phi(t))*cos(theta(t))],
[                                 0]])

In [49]:
# --- Generalized input force vector ---
tau = sp.Matrix(sp.symbols('tau1:7')).reshape(6, 1)
tau

Matrix([
[tau1],
[tau2],
[tau3],
[tau4],
[tau5],
[tau6]])

In [50]:
# --- Time derivative of velocity ---
nu_dot = sp.Matrix(sp.symbols('udot vdot wdot pdot qdot rdot')).reshape(6, 1)
nu_dot

Matrix([
[udot],
[vdot],
[wdot],
[pdot],
[qdot],
[rdot]])

In [51]:
# --- Full rigid-body dynamic equation ---
dyn_eq = MRB * nu_dot + CRB * nu + g_eta - tau
dyn_eq

Matrix([
[                                                                                                                           m*qdot*z_g - m*rdot*y_g + m*udot - m*x_g*q(t)*v(t) - m*x_g*r(t)*w(t) + m*q(t)*w(t) - m*r(t)*v(t) - tau1 + (m*y_g*v(t) + m*z_g*w(t))*p(t)],
[                                                                                                                          -m*pdot*z_g + m*rdot*x_g + m*vdot - m*y_g*p(t)*u(t) - m*y_g*r(t)*w(t) - m*p(t)*w(t) + m*r(t)*u(t) - tau2 + (m*x_g*u(t) + m*z_g*w(t))*q(t)],
[                                                                                                                     g*m + m*pdot*y_g - m*qdot*x_g + m*wdot - m*z_g*p(t)*u(t) - m*z_g*q(t)*v(t) + m*p(t)*v(t) - m*q(t)*u(t) - tau3 + (m*x_g*u(t) + m*y_g*v(t))*r(t)],
[            I_xx*pdot - I_xy*qdot - I_xz*rdot + g*m*z_g*sin(theta(t)) - m*vdot*z_g + m*wdot*y_g + m*y_g*u(t)*v(t) + m*z_g*u(t)*w(t) - tau4 + (-m*y_g*v(t) - m*z_g*w(t))*u(t) + (I_xy*p(t) - I_yy*q(t) + I

In [52]:
# Solve for nu_dot symbolically
nu_dot_expr = MRB.inv() * (tau - CRB * nu - g_eta)
nu_dot_expr

Matrix([
[    (m*x_g*q(t)*v(t) + m*x_g*r(t)*w(t) - m*q(t)*w(t) + m*r(t)*v(t) + tau1 - (m*y_g*v(t) + m*z_g*w(t))*p(t))*(I_xx*I_yy*I_zz - I_xx*I_yy*m*x_g**2 - I_xx*I_yz**2 - I_xx*I_zz*m*x_g**2 + I_xx*m**2*x_g**4 - I_xy**2*I_zz + I_xy**2*m*x_g**2 - 2*I_xy*I_xz*I_yz + 2*I_xy*I_yz*m*x_g*z_g + 2*I_xy*I_zz*m*x_g*y_g - 2*I_xy*m**2*x_g**3*y_g - I_xz**2*I_yy + I_xz**2*m*x_g**2 + 2*I_xz*I_yy*m*x_g*z_g + 2*I_xz*I_yz*m*x_g*y_g - 2*I_xz*m**2*x_g**3*z_g - I_yy*I_zz*m*y_g**2 - I_yy*I_zz*m*z_g**2 + I_yy*m**2*x_g**2*y_g**2 + I_yz**2*m*y_g**2 + I_yz**2*m*z_g**2 - 2*I_yz*m**2*x_g**2*y_g*z_g + I_zz*m**2*x_g**2*z_g**2)/(I_xx*I_yy*I_zz*m - I_xx*I_yy*m**2*x_g**2 - I_xx*I_yy*m**2*y_g**2 - I_xx*I_yz**2*m + 2*I_xx*I_yz*m**2*y_g*z_g - I_xx*I_zz*m**2*x_g**2 - I_xx*I_zz*m**2*z_g**2 + I_xx*m**3*x_g**4 + I_xx*m**3*x_g**2*y_g**2 + I_xx*m**3*x_g**2*z_g**2 - I_xy**2*I_zz*m + I_xy**2*m**2*x_g**2 + I_xy**2*m**2*y_g**2 - 2*I_xy*I_xz*I_yz*m + 2*I_xy*I_xz*m**2*y_g*z_g + 2*I_xy*I_yz*m**2*x_g*z_g + 2*I_xy*I_zz*m**2*x_g*y_g - 2

In [53]:
x_dot = sp.Matrix.vstack(d_eta, nu_dot_expr)
x_dot

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              